In [ ]:
# generate qr code
import qrcode
import sys
import numpy as np

def generate_qr_code(data, filename=None, version=None, error_correction=qrcode.constants.ERROR_CORRECT_H, border=4, mask_logo=True):
    qr = qrcode.QRCode(
        version=version,  # 1-40, None
        error_correction=error_correction,
        box_size=20,  # pixels per box
        border=border,  # boxes of border
    )
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image()
    if filename is not None:
        img.save(filename)

    if mask_logo:
        # add a white square at the center of the QR code, use box_size=20
        img = img.convert("RGB")
        img = np.array(img)
        height, width, _ = img.shape
        # add a white square at the center
        img[height//2-70:height//2+70, width//2-70:width//2+70] = 255, 255, 255
        img = Image.fromarray(img)
    return img

# function to replace white parts of qr code with noise
def add_noise_to_qr_code(qr_code_image, noise_level=0.2, border_noise_level=1.0):
    """
    Adds color noise to a QR code image.

    - Replaces the borders around the QR code with color noise proportional to border_noise_level.
    - Adds color noise to the black and white sections inside the QR code proportional to noise_level.
    - If noise_level or border_noise_level is 0, no change is made to that area.
    - If noise_level or border_noise_level is 1, the area is completely replaced with noise.

    Parameters:
    - qr_code_image: PIL Image object of the QR code.
    - noise_level: float (0 to 1), amount of noise to add to the inner QR code.
    - border_noise_level: float (0 to 1), amount of noise to add to the border.
    """
    # Convert image to RGB and numpy array (float32 for precision)
    qr_array = np.array(qr_code_image.convert('RGB'), dtype=np.float32)

    # Get dimensions
    height, width, channels = qr_array.shape

    # Convert to grayscale for thresholding
    qr_gray = np.array(qr_code_image.convert('L'))

    # Threshold to create a binary mask (True for white pixels, False for black pixels)
    threshold = 128
    binary_mask = qr_gray > threshold

    # Find the bounding box of the QR code modules (the inner area)
    coords = np.column_stack(np.where(binary_mask == False))  # False corresponds to black pixels
    if coords.size == 0:
        raise ValueError("QR code modules not found in the image.")

    top_left = coords.min(axis=0)
    bottom_right = coords.max(axis=0)

    # Create masks for border and inner areas
    border_mask = np.ones((height, width), dtype=bool)
    inner_mask = np.zeros((height, width), dtype=bool)

    # Define the inner QR code area
    inner_mask[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1] = True
    border_mask[inner_mask] = False  # Border is where inner_mask is False

    # Apply noise to border area
    if border_noise_level > 0:
        # Generate color noise for border area
        border_noise = np.random.randint(0, 256, size=(height, width, channels), dtype=np.uint8)
        border_noise = border_noise.astype(np.float32)
        # Blend the original border area with noise based on border_noise_level
        qr_array[border_mask] = (1 - border_noise_level) * qr_array[border_mask] + border_noise_level * border_noise[border_mask]
    # Else, border_noise_level == 0, so no change to border area

    # Apply noise to inner area
    if noise_level > 0:
        # Generate color noise for inner area
        inner_noise = np.random.randint(0, 256, size=(height, width, channels), dtype=np.uint8)
        inner_noise = inner_noise.astype(np.float32)
        # Blend the original inner area with noise based on noise_level
        qr_array[inner_mask] = (1 - noise_level) * qr_array[inner_mask] + noise_level * inner_noise[inner_mask]
    # Else, noise_level == 0, so no change to inner area

    # Convert back to uint8
    qr_array = np.clip(qr_array, 0, 255).astype(np.uint8)

    # Convert back to PIL Image
    noisy_qr_code = Image.fromarray(qr_array, 'RGB')

    return noisy_qr_code

# Prompt Generator
import random

# Template: "A QR code {verb} into a {adjective} {location}, {artstyle}"

def generate_prompt():
    verbs = [
        'integrated',
        'blended',
        'embedded',
        'merged',
        'incorporated',
        'interwoven',
        'woven',
        'fused',
        'engraved',
        'imprinted'
    ]
    
    adjectives = [
        'futuristic',
        'ancient',
        'mystical',
        'vibrant',
        'serene',
        'abstract',
        'surreal',
        'majestic',
        'ethereal',
        'dynamic',
        'colorful',
        'monochrome',
        'minimalist',
        'intricate',
        'ornate'
    ]
    
    locations = [
        'cityscape',
        'forest',
        'mountain landscape',
        'underwater scene',
        'space nebula',
        'desert',
        'ocean',
        'galaxy',
        'garden',
        'skyline',
        'countryside',
        'rainforest',
        'ice cave',
        'ancient temple',
        'futuristic metropolis'
    ]
    
    artstyles = [
        'digital art',
        'oil painting',
        'watercolor',
        'pencil sketch',
        'cyberpunk style',
        'steampunk aesthetic',
        'fantasy art',
        'minimalist design',
        'photorealistic rendering',
        'pop art',
        'impressionist painting',
        'surrealism',
        'abstract expressionism',
        'low-poly art',
        'graffiti style'
    ]

    #verb = random.choice(verbs)
    verb = 'integrated'
    adjective = random.choice(adjectives)
    location = random.choice(locations)
    artstyle = random.choice(artstyles)

    prompt = f"A QR code, cleverly {verb} into a {adjective} {location}, {artstyle}, 3d rendered"
    # Generate filename that includes information from prompt but short and unique
    filename = f"{adjective}_{location}_{artstyle}"
    return prompt, filename

import torch
from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel, DDIMScheduler
from diffusers.utils import load_image
from PIL import Image
#from realesrgan import RealESRGAN

def resize_for_condition_image(input_image: Image.Image, resolution: int):
    input_image = input_image.convert("RGB")
    W, H = input_image.size
    k = float(resolution) / min(H, W)
    H = int(H * k)
    W = int(W * k)
    H = (H // 64) * 64
    W = (W // 64) * 64
    img = input_image.resize((W, H), resample=Image.LANCZOS)
    return img

from PIL import ImageOps

def run_diffusion_on_qr_code(
    image_of_qr,
    prompt,
    output_filename,
    model_id='dreamlike-art/dreamlike-diffusion-1.0',
    controlnet_model_id='DionTimmer/controlnet_qrcode-control_v1p_sd15',
    strength=0.9,
    guidance_scale=20,
    controlnet_conditioning_scale=1.5,
    num_inference_steps=150,
    resolution=768,
    seed=11111,
    noise_level=0.5,
    border_noise_level=1.0,
    verbose=False,
    invert_colors=False,
    upscale_resolution=None
):
    
    output_filename = f'output_images/{output_filename}'
    if verbose:
        print(f'Running qr code diffusion with prompt: {prompt}')
    # Check for available devices
    if torch.cuda.is_available():
        device = torch.device("cuda")
        torch_dtype = torch.float16  # CUDA supports float16 for performance
        if verbose:
            print("Using CUDA device")
    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        device = torch.device("mps")
        torch_dtype = torch.float32  # MPS supports float32
        if verbose:
            print("Using MPS device")
    else:
        device = torch.device("cpu")
        torch_dtype = torch.float32  # CPU uses float32
        if verbose:
            print("Using CPU device")

    # Load ControlNet model
    controlnet = ControlNetModel.from_pretrained(
        controlnet_model_id,
        torch_dtype=torch_dtype
    ).to(device)

    # Load the pipeline
    pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
        'models/animerge_v23',
        controlnet=controlnet,
        safety_checker=None,
        torch_dtype=torch_dtype
    ).to(device)

    # Memory optimization
    if device.type == 'cuda':
        # Disable xFormers due to incompatibility
        if verbose:
            print("xFormers not compatible with this GPU; enabling attention slicing instead.")
        pipe.enable_attention_slicing()
    elif device.type == 'mps':
        pipe.enable_attention_slicing()
    else:
        pipe.enable_attention_slicing()

    # Optional: Enable other memory optimizations
    pipe.enable_model_cpu_offload()

    pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

    # Resize the QR code image (condition image)
    condition_image = resize_for_condition_image(image_of_qr, resolution)


    # Prepare the initial image
    init_image = image_of_qr.copy()
    init_image = add_noise_to_qr_code(init_image, noise_level, border_noise_level)
    init_image = resize_for_condition_image(init_image, resolution)

    if invert_colors:
        init_image = init_image.convert("RGB")
        init_image = ImageOps.invert(init_image)

    # Set a random seed for reproducibility
    generator = torch.Generator(device=device).manual_seed(seed)

    # Generate the image
    result = pipe(
        prompt=prompt,
        negative_prompt="ugly, disfigured, low quality, blurry",
        image=init_image,
        control_image=condition_image,
        width=condition_image.width,
        height=condition_image.height,
        guidance_scale=guidance_scale,
        controlnet_conditioning_scale=controlnet_conditioning_scale,
        generator=generator,
        strength=strength,
        num_inference_steps=num_inference_steps,
    )

    final_image = result.images[0]

    # Upscaling with RealESRGAN
    if upscale_resolution is not None:
        if verbose:
            print(f"Upscaling image to resolution: {upscale_resolution}")

        model = RealESRGAN(device, scale=4)
        model.load_weights('RealESRGAN_x4.pth')  # Make sure to download weights

        # Calculate scaling factor to reach desired upscale resolution
        scale_factor = max(upscale_resolution / final_image.width, upscale_resolution / final_image.height)
        intermediate_size = (int(final_image.width * scale_factor), int(final_image.height * scale_factor))

        # Perform AI upscaling
        final_image = model.predict(final_image.resize(intermediate_size, resample=Image.LANCZOS))

        # Crop or resize exactly to upscale_resolution if necessary
        final_image = final_image.resize((upscale_resolution, upscale_resolution), Image.LANCZOS)

    final_image.save(output_filename)

    if verbose:
        print(f"Image saved to {output_filename}")

: 

In [6]:
import qrcode

message = 'https://www.linkedin.com/in/till-zacher/'
qr_filename = 'LinkedIn.png' # qr will not be saved to disk
image_of_qr = generate_qr_code(message, filename=qr_filename, error_correction=qrcode.constants.ERROR_CORRECT_H, border=10, mask_logo=True)
print(f"Generated QR code with message: {message}")
# show image in jupyter notebook
#image_of_qr.show()

# Generate the prompt
prompt, prompt_filename = generate_prompt()
print(f"Generated Prompt: {prompt}")

Generated QR code with message: https://www.linkedin.com/in/till-zacher/
Generated Prompt: A QR code, cleverly integrated into a minimalist garden, cyberpunk style, 3d rendered


In [7]:

# Run once
import random
strength = 0.9
guidance_scale = 7  # how much to focus on the prompt
controlnet_conditioning_scale = 1.2 # how much to focus on the condition image
resolution = 1024
seed = 384
#seed = random.randint(0, 100000)
print(f"Using seed {seed}")
num_steps = 100 # 30
noise_level = 0.6
border_noise_level = 0.3
verbose = True
invert_colors = True

#prompt_override = 'magical computer hardware, electronics, futuristic, abstract'
prompt_override = 'secret chamber inside an ice cave'
#
# prompt_override = None
if prompt_override is not None:
    prompt = prompt_override
else:
    prompt, prompt_filename = generate_prompt()

#output_filename = f'{message}_{prompt_filename}_s{strength}_gs{guidance_scale}_ccs{controlnet_conditioning_scale}.png'
#output_filename = f'{message}_{prompt_filename}.png'
output_filename = 'Test.png'

run_diffusion_on_qr_code(image_of_qr=image_of_qr, prompt=prompt, strength=strength, guidance_scale=guidance_scale, controlnet_conditioning_scale=controlnet_conditioning_scale, resolution=resolution, seed=seed, output_filename=output_filename, num_inference_steps=num_steps, noise_level=noise_level, verbose=verbose, border_noise_level=border_noise_level, invert_colors=invert_colors)

Using seed 384
Running qr code diffusion with prompt: secret chamber inside an ice cave
Using CUDA device


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch models/animerge_v23\vae: Error no file named diffusion_pytorch_model.safetensors found in directory models/animerge_v23\vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch models/animerge_v23\unet: Error no file named diffusion_pytorch_model.safetensors found in directory models/animerge_v23\unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only 

xFormers not compatible with this GPU; enabling attention slicing instead.


RuntimeError: Numpy is not available

In [ ]:
import itertools
import os
import random

# Define the ranges for each parameter
strength_values = [0.9]  # Example values for strength
guidance_scale_values = [7, 15]  # Example values for guidance_scale
controlnet_conditioning_scale_values = [1.9, 2.6]  # Example values for controlnet_conditioning_scale
resolution_values = [1024]  # Example resolution
# generate integer seed range
seed_values = [random.randint(0, 100000) for i in range(8)]  # Example values for seed
noise_level_values = [0.3, 0.6]  # Example values for noise_level
border_noise_level_values = [0.3, 0.6]
num_inference_steps_values = [30]  # Example values for num_inference_steps
invert_colors_values = [True]

# Ensure the output directory exists
output_dir = "data_science_test"
os.makedirs(output_dir, exist_ok=True)

# Generate all combinations of parameters
parameter_combinations = list(itertools.product(
    strength_values,
    guidance_scale_values,
    controlnet_conditioning_scale_values,
    resolution_values,
    seed_values,
    noise_level_values,
    border_noise_level_values,
    num_inference_steps_values,
    invert_colors_values
))

print(f"Total combinations to process: {len(parameter_combinations)}")

# Loop over each combination and run the diffusion process
for idx, (strength, guidance_scale, controlnet_conditioning_scale, resolution, seed, noise_level, border_noise_level, num_steps, invert_colors) in enumerate(parameter_combinations):

    # If using a prompt override, ensure it's set
    #prompt_override = 'A busy chinese market scene, has a QR code cleverly embedded into the scene, 3D, animated'
    prompt_override = 'magical computer hardware, electronics, futuristic, abstract'
    if prompt_override is not None:
        prompt = prompt_override
        prompt_filename = "prompt_override"
    else:
        # Generate the prompt
        prompt, prompt_filename = generate_prompt()

    # Generate a unique output filename for each combination
    output_filename = os.path.join(
        output_dir,
        f"{message}_{prompt_filename}_s{strength}_gs{guidance_scale}_ccs{controlnet_conditioning_scale}_r{resolution}_seed{seed}_nl{noise_level}_bnl{border_noise_level}_steps{num_steps}_inv{invert_colors}.png"
    )

    # Run the diffusion process
    run_diffusion_on_qr_code(
        image_of_qr=image_of_qr,
        prompt=prompt,
        strength=strength,
        guidance_scale=guidance_scale,
        controlnet_conditioning_scale=controlnet_conditioning_scale,
        resolution=resolution,
        seed=seed,
        output_filename=output_filename,
        num_inference_steps=num_steps,
        noise_level=noise_level,
        border_noise_level=border_noise_level,
        invert_colors=invert_colors,
        verbose=False
    )

Total combinations to process: 128


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch models/animerge_v23\vae: Error no file named diffusion_pytorch_model.safetensors found in directory models/animerge_v23\vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch models/animerge_v23\unet: Error no file named diffusion_pytorch_model.safetensors found in directory models/animerge_v23\unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only 

RuntimeError: Numpy is not available

In [11]:
import numpy